In [2]:
import pandas as pd
import os
import json
import sys
from tqdm import tqdm
from itertools import combinations

In [3]:
sys.path.append("/home/aberhe/Projects/SANTAL/ASLR/src/")

In [4]:
import config as cf
import jsonlines

In [4]:
cf.PATH_RESULTS

'/home/aberhe/Projects/SANTAL/ASLR/Results'

In [ ]:
os.path

In [5]:
path_raw_jsonl="/home/aberhe/Projects/SANTAL/ASLR/Results/species_annotation.jsonl"

In [6]:
d=pd.read_json(os.path.join(cf.PATH_RESULTS,'species_annotation.jsonl.json'), lines=True)

In [7]:
d.head()

,text,Sent_ID,Entities
0,Uterine infection is an important disease affe...,PMC4312817_0,"{'text': ['Uterine infection', 'infertility.1'..."
1,Bacteria growing in the presence of E. arvense...,PMC3495101_164,"{'text': ['E. arvense', 'H. glabra', 'E. coli'..."
2,Identified taxa of the oronasal habitat includ...,PMC3797391_87,"{'text': ['Dialister spp.', 'Fusobacterium nuc..."
3,The lingonberry extract had the greatest amoun...,PMC3495101_206,"{'text': ['lingonberry', 'plants', 'B. pendula..."
4,Real-time PCR analysis was performed by using ...,PMC5769758_109,"{'text': ['Lactobacillus acidophilus', 'Strept..."


In [8]:
len(set(d.text))

3840

In [9]:
d.shape

(6966, 3)

In [ ]:
df_unique_sentences=[]

In [8]:
#df_species_disease_chemical2.to_csv(os.path.join(cf.PATH_RESULTS,"Species_Disease_Chemical_extracted_2_Entities_02_11.csv"))
df_species_disease_chemical2=pd.read_csv(os.path.join(cf.PATH_RESULTS,"Species_Disease_Chemical_extracted_2_Entities_02_11.csv"))
df_species_disease_chemical2=df_species_disease_chemical2.drop(columns=["Unnamed: 0","Unnamed: 0.1"])

In [10]:
df_species_disease_chemical2.head()

,sent_id,sentence,entity,start_chunck,end_chunk,label,score,PMC_ID
0,PMC10000417_0,The term “ peri-implant diseases ” refers to t...,peri-implant diseases,10,31,Disease,0.724,PMC10000417
1,PMC10000417_0,The term “ peri-implant diseases ” refers to t...,mucositis,180,189,Disease,0.537,PMC10000417
2,PMC10000417_0,The term “ peri-implant diseases ” refers to t...,peri-implantitis,194,210,Disease,0.969,PMC10000417
3,PMC10000417_1,"In 2008 , mucositis was defined as inflammatio...",mucositis,9,18,Disease,0.993,PMC10000417
4,PMC10000417_1,"In 2008 , mucositis was defined as inflammatio...",inflammation,34,46,Disease,0.841,PMC10000417


In [8]:
def construct_pair_entities(df):
    sent_id=df.sent_id[0]
    df_2Entities=pd.DataFrame(columns=["PMC_ID","sent_id","sentence","entity1","entity2","label1","label2"])
    for i in tqdm(range(1,len(df))):
        try:
            if df.sent_id[i-1]==df.sent_id[i] and df.entity[i-1]!=df.sent_id[i]:
                res=[df.PMC_ID[i],df.sent_id[i],df.sentence[i],
                    df.entity[i-1],df.entity[i],df.label[i-1],
                    df.label[i]]
                df_2Entities.loc[len(df_2Entities)]=res
        except:
            continue
            #print(f"index {i} have a problem")
    return df_2Entities

In [13]:
df_2Entities=construct_pair_entities(df=df_species_disease_chemical2)

  0%|          | 0/424752 [00:00<?, ?it/s]

100%|██████████| 424752/424752 [46:00<00:00, 153.86it/s]


In [14]:
df_2Entities.shape

(230518, 7)

In [7]:
#df_2Entities.to_csv(os.path.join(cf.PATH_RESULTS,"Species_Disease_Chemical_extracted_2_pair_Entities_02_11.csv"))
df_2Entities=pd.read_csv(os.path.join(cf.PATH_RESULTS,"Species_Disease_Chemical_extracted_2_pair_Entities_02_11.csv"))
df_2Entities=df_2Entities.drop(columns=["Unnamed: 0"])

In [24]:
df_2Entities12=df_2Entities[df_2Entities["label1"]!=df_2Entities["label2"]]
df_2Entities12=df_2Entities12.reset_index()

In [9]:
df_2Entities12.head()

,PMC_ID,sent_id,sentence,entity1,entity2,label1,label2
30,PMC10000417,PMC10000417_60,Film proteins contain various amino acid seque...,amino acid,bacteria,Chemical,Species
32,PMC10000417,PMC10000417_64,Oxygen is gradually consumed by facultative ae...,Oxygen,Fusobacterium nucleatum,Chemical,Species
47,PMC10000417,PMC10000417_81,are frequently isolated from peri-implant lesi...,lesions,Campylobacter rectus,Disease,Species
70,PMC10000417,PMC10000417_136,"In a bactericidal modality , CO2 , diode , erb...",YSGG,peri-implant diseases,Chemical,Disease
76,PMC10000417,PMC10000417_142,Slow-release antibacterial agents include chlo...,chlorhexidine,periodontitis,Chemical,Disease


In [10]:
df_2Entities12.shape

(63531, 7)

In [26]:
print(df_2Entities12["sentence"][178073])

Fifteen cases were considered cryptogenic.Table 1Summary of case reports identifiedAgeSexOrganism ( specimen cultured ) TreatmentSingle or multiple abscess ( es ) Presumed source1 [ 35 ] 21M F. nucleatum ( abscess aspirate )- Drainage of the two largest abscesses ( R ).- Piperacillin / tazobactam ( UTP )- Ertapenem ( 8 weeks ) MultipleOropharynx ( PF ) 2 [ 36 ] 48M F. nucleatum ( abscess aspirate )- Drainage of abscess ( R )- Ceftriaxone and metronidazole ( 3 days )- IV penicillin and metronidazole ( 10 weeks ) MultipleOropharynx ( PF ) 3 [ 37 ] 23M F. nucleatum ( blood culture and abscess aspirate )- Drainage of abscess ( R )- Piperacillin / tazobactam ( UTP )- Ertapenem ( 6 weeks ) MultipleUnidentified4 [ 38 ] 58M F. nucleatum ( abscess aspirate )- Drainage of pelvic abscess ( S )- Defunctioning colostomy - Ertapenem ( 6 weeks )- Low molecular weight heparin ( 2 weeks ) MultipleLower GI tract ( diverticulitis ) 5 [ 39 ] 48M F. nucleatum ( abscess aspirate )- Drainage of two largest a

In [32]:
indices_selct=[i for i in range(len(df_2Entities12)) if len(df_2Entities12["sentence"][i].split()) >10 and len(df_2Entities12["sentence"][i].split())<20]

In [33]:
selected_rows_iloc=df_2Entities12.iloc[indices_selct]

In [34]:
selected_rows_iloc=selected_rows_iloc.reset_index()

In [36]:
selected_rows_iloc.drop

,level_0,index,PMC_ID,sent_id,sentence,entity1,entity2,label1,label2
0,5,78,PMC10000417,PMC10000417_144,Chlorhexidine gel xanthan is a product indicat...,Chlorhexidine,periodontitis,Chemical,Disease
1,14,128,PMC10000417,PMC10000417_220,Chlorhexidine gel xanthan is a product indicat...,Chlorhexidine,periodontitis,Chemical,Disease
2,49,259,PMC10000660,PMC10000660_308,[ 61 ] observed that the genus Paracoccus was ...,patients,BPH,Species,Disease
3,106,525,PMC10002314,PMC10002314_214,The present study identified 32 genera that co...,opportunistic infections,humans,Disease,Species
4,114,551,PMC10002314,PMC10002314_286,The present study identified 32 genera that co...,opportunistic infections,humans,Disease,Species
...,...,...,...,...,...,...,...,...,...
6728,63457,230254,PMC5866888,PMC5866888_366,Honkanen and colleagues clearly demonstrated a...,patients,T1DM,Species,Disease
6729,63490,230343,PMC5866915,PMC5866915_130,( 2006 ) showed that HPV infection is less pre...,HPV infection,HPV,Disease,Species
6730,63501,230356,PMC5866915,PMC5866915_149,We detected a negative association between oro...,oropharyngeal HPV infection,HPV,Disease,Species
6731,63507,230366,PMC5866915,PMC5866915_160,"Koilocitosis , a marker of nuclear changes con...",HPV infection,HPV,Disease,Species


In [38]:
remove_list=["patients","patient","children","child"]

In [9]:
selected_rows_iloc_rmvd=selected_rows_iloc[(~selected_rows_iloc["entity1"].isin(remove_list)) & (~selected_rows_iloc["entity2"].isin(remove_list))]

NameError: name 'selected_rows_iloc' is not defined

In [42]:
selected_rows_iloc_rmvd=selected_rows_iloc_rmvd.reset_index()

,level_0,index,PMC_ID,sent_id,sentence,entity1,entity2,label1,label2
0,5,78,PMC10000417,PMC10000417_144,Chlorhexidine gel xanthan is a product indicat...,Chlorhexidine,periodontitis,Chemical,Disease
1,14,128,PMC10000417,PMC10000417_220,Chlorhexidine gel xanthan is a product indicat...,Chlorhexidine,periodontitis,Chemical,Disease
3,106,525,PMC10002314,PMC10002314_214,The present study identified 32 genera that co...,opportunistic infections,humans,Disease,Species
4,114,551,PMC10002314,PMC10002314_286,The present study identified 32 genera that co...,opportunistic infections,humans,Disease,Species
5,182,880,PMC10003408,PMC10003408_36,The cultures of pus from skin lesions confirme...,skin lesions,Staphylococcus aureus,Disease,Species
...,...,...,...,...,...,...,...,...,...
6725,63364,230065,PMC5862454,PMC5862454_340,Low intake was associated with higher prevalen...,caries,S. mutans,Disease,Species
6729,63490,230343,PMC5866915,PMC5866915_130,( 2006 ) showed that HPV infection is less pre...,HPV infection,HPV,Disease,Species
6730,63501,230356,PMC5866915,PMC5866915_149,We detected a negative association between oro...,oropharyngeal HPV infection,HPV,Disease,Species
6731,63507,230366,PMC5866915,PMC5866915_160,"Koilocitosis , a marker of nuclear changes con...",HPV infection,HPV,Disease,Species


In [16]:
selected_rows_iloc_rmvd=pd.read_csv("Selected_Sentences_words_10_20_Species_Disease_Chemical_extracted_2_pair_Entities_06_11_Patiens_child_Removed.csv")

In [17]:
len(selected_rows_iloc_rmvd)

5297

In [10]:
def getEntity_start_end(text="",entity=""):
    #print(text)
    #print(entity)
    text=text.strip().replace("( ","(").replace(" )",")")
    entity=entity.strip().replace("( ","(").replace(" )",")")
    start_index = text.find(entity)
    if start_index != -1:
        end_index = start_index + len(entity) - 1
        return start_index, end_index
    else:
        return None



In [13]:
getEntity_start_end(text=selected_rows_iloc_rmvd["sentence"][0],entity=selected_rows_iloc_rmvd["entity1"][0])

(0, 12)

In [12]:
start,end=getEntity_start_end(text=selected_rows_iloc_rmvd["sentence"][0],entity=selected_rows_iloc_rmvd["entity1"][0])
start2,end2=getEntity_start_end(text=selected_rows_iloc_rmvd["sentence"][0],entity=selected_rows_iloc_rmvd["entity2"][0])
dataDict={"text":selected_rows_iloc_rmvd["sentence"][0],"relation":"","spans":[{"start":start,"end":end,"label":selected_rows_iloc_rmvd["label1"][0]},{"start":start2,"end":end2,"label":selected_rows_iloc_rmvd["label2"][0]}]}

In [13]:
dataDict["spans"].append({"start":start,"end":end,"label":selected_rows_iloc_rmvd["label1"][0]})
dataDict["spans"].append({"start":start2,"end":end2,"label":selected_rows_iloc_rmvd["label2"][0]})

In [14]:
dataDict

{'text': 'Chlorhexidine gel xanthan is a product indicated for the topical treatment of periodontitis and peri-implantitis .',
 'relation': '',
 'spans': [{'start': 0, 'end': 12, 'label': 'Chemical'},
  {'start': 78, 'end': 90, 'label': 'Disease'},
  {'start': 0, 'end': 12, 'label': 'Chemical'},
  {'start': 78, 'end': 90, 'label': 'Disease'}]}

In [15]:
selected_rows_iloc_rmvd.columns

Index(['Unnamed: 0', 'level_0', 'index', 'PMC_ID', 'sent_id', 'sentence',
       'entity1', 'entity2', 'label1', 'label2'],
      dtype='object')

In [12]:
selected_rows_iloc_rmvd=selected_rows_iloc_rmvd.drop(columns=['Unnamed: 0', 'level_0', 'index'])
selected_rows_iloc_rmvd=selected_rows_iloc_rmvd.reset_index()

In [17]:
selected_rows_iloc_rmvd["sent_id"][18]

'PMC10004182_258'

In [27]:
import re
import spacy
def clean_text(text):
    pattern = r'(\(|\[|\{)\s*([^(\s*)\[\s*{\s*}\s*\])]*?)\s*(\)|\]|\})' 
    pattern2=r'\s*([!@#$%^&*_+=:;"\'<>,.?/\|\\~`])\s*'

    text=re.sub(pattern,r'\1\2\3',text)
    text=re.sub(pattern2,r'\1 ',text)
    text=text.replace(" .",".").replace(". ",".")
    return text

def split_text_with_positions(text):
    tokens = []
    start = 0
    for i, char in enumerate(text):
        if char == " ":
            word = text[start:i]
            tokens.append({"text": word, "start": start, "end": i, "id": len(tokens),"ws": True})
            start = i + 1

    if start < len(text):
        word = text[start:]
        tokens.append({"text": word, "start": start, "end": len(text), "id": len(tokens),"ws": True})

    return tokens

nlp = spacy.load("en_core_web_sm")
def spacy_tokenizer(text=""):
    no_ws_list=[".","-","+","_"]
    tokenized_text = nlp(text)
    tokens=[]
    for i,token in enumerate(tokenized_text):
        tokens.append({"text": token.text, "start": token.idx, "end": token.idx+len(token.text)-1, "id": i,"ws":False})
        """
        if token.text not in no_ws_list:
            if token.text in ["[","(","{"]:
                tokens.append({"text": token.text, "start": token.idx, "end": token.idx+len(token.text)-1, "id": i,"ws":False})
            elif token.text in ["]",")","}",","]:
                tokens[-1]["ws"]=False
                tokens.append({"text": token.text, "start": token.idx, "end": token.idx+len(token.text)-1, "id": i,"ws":True})
            else:
                tokens.append({"text": token.text, "start": token.idx, "end": token.idx+len(token.text)-1, "id": i,"ws":True})
        else:
            tokens[-1]["ws"]=False
            tokens.append({"text": token.text, "start": token.idx, "end": token.idx+len(token.text)-1, "id": i,"ws":False})

        """
    return tokens

def getSpanToken_indices(tokens=[],span=""):
    span_tokens=spacy_tokenizer(span)
    token_list=[tokens[i]["text"] for i in range(len(tokens))]
    token_start=token_list.index(span_tokens[0]["text"])
    if len(span_tokens)>1:
        token_end=token_list.index(span_tokens[-1]["text"])
    else:
        token_end=token_start
    
    return token_start,token_end

/opt/conda/lib/python3.11/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.6.0) was trained with spaCy v3.6.0 and may not be 100% compatible with the current version (3.7.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [20]:
getSpanToken_indices(tokens=spacy_tokenizer(clean_text(selected_rows_iloc_rmvd["sentence"][0])),span=clean_text(selected_rows_iloc_rmvd["entity1"][0]))

('Chlorhexidine', 0, 0)

In [18]:
spacy_tokenizer(clean_text(selected_rows_iloc_rmvd["entity1"][0]))

[{'text': 'Chlorhexidine', 'start': 0, 'end': 12, 'id': 0, 'ws': True}]

In [28]:
def find_closest_indices(list_toke_dict, given_start, given_end):
    list_of_starts=[list_toke_dict[i]["start"] for i in range(len(list_toke_dict))]
    list_of_ends=[list_toke_dict[i]["end"] for i in range(len(list_toke_dict))]
    entity_token_start = None
    entity_token_end = None
    min_start_diff = float("inf")
    min_end_diff = float("inf")

    for i, start in enumerate(list_of_starts):
        start_diff = abs(given_start - start)
        if start_diff < min_start_diff:
            min_start_diff = start_diff
            entity_token_start = i

    for i, end in enumerate(list_of_ends):
        end_diff = abs(given_end - end)
        if end_diff < min_end_diff:
            min_end_diff = end_diff
            entity_token_end = i

    return entity_token_start, entity_token_end

In [15]:
start,end=getEntity_start_end(text=selected_rows_iloc_rmvd["sentence"][1],entity="Chlorhexidine gel xanthan")#selected_rows_iloc_rmvd["entity2"][1])

In [16]:
start,end,selected_rows_iloc_rmvd["entity1"][1],selected_rows_iloc_rmvd["entity2"][1]

(0, 24, 'Chlorhexidine', 'periodontitis')

In [56]:
selected_rows_iloc_rmvd["sentence"][0]

'Chlorhexidine gel xanthan is a product indicated for the topical treatment of periodontitis and peri-implantitis .'

In [17]:
text=selected_rows_iloc_rmvd["sentence"][0]#selected_rows_iloc_rmvd["sentence"][18]
print(text)
print(clean_text(text))
text=clean_text(text)

Chlorhexidine gel xanthan is a product indicated for the topical treatment of periodontitis and peri-implantitis .
Chlorhexidine gel xanthan is a product indicated for the topical treatment of periodontitis and peri-implantitis.


In [20]:
selected_rows_iloc_rmvd["entity1"][18]

'Ca2+'

In [13]:
from tqdm import tqdm

In [29]:
sent_id_priv=selected_rows_iloc_rmvd["sent_id"][0]
dict_list=[]
text=selected_rows_iloc_rmvd["sentence"][0]
text=clean_text(text)
entity1=clean_text(text=selected_rows_iloc_rmvd["entity1"][0])
entity2=clean_text(text=selected_rows_iloc_rmvd["entity2"][0])
#start,end=getEntity_start_end(text=text,entity=entity1)
#start2,end2=getEntity_start_end(text=text,entity=entity2)
tokens=spacy_tokenizer(text=text)#split_text_with_positions(text=text)
#token_start,token_end=find_closest_indices(tokens, start,end)
#token_start2,token_end2=find_closest_indices(tokens, start2,end2)
token_start,token_end=getSpanToken_indices(tokens,span=entity1)
token_start2,token_end2=getSpanToken_indices(tokens,span=entity2)
start=tokens[token_start]["start"]
end=tokens[token_end]["end"]
start2=tokens[token_start2]["start"]
end2=tokens[token_end2]["end"]
l=1
meta={"PMC_ID":selected_rows_iloc_rmvd["PMC_ID"][0],"Sentence_id":selected_rows_iloc_rmvd["sent_id"][0],"index":0,"line":l}
dataDict={"meta":meta,"text":text,"tokens":tokens,"spans":[{"start":start,"end":end,"token_start":token_start,"token_end":token_end,"label":selected_rows_iloc_rmvd["label1"][0],"text":entity1},{"start":start2,"end":end2,"token_start":token_start2,"token_end":token_end2,"label":selected_rows_iloc_rmvd["label2"][0],"text":entity2}]}
for i in tqdm(range(1,len(selected_rows_iloc_rmvd))):
    try:
        if sent_id_priv==selected_rows_iloc_rmvd["sent_id"][i] or text==clean_text(selected_rows_iloc_rmvd["sentence"][i]):
            entity1=clean_text(text=selected_rows_iloc_rmvd["entity1"][i])
            entity2=clean_text(text=selected_rows_iloc_rmvd["entity2"][i])
            start,end=getEntity_start_end(text=text,entity=entity1)
            start2,end2=getEntity_start_end(text=text,entity=entity2)
            #token_start,token_end=find_closest_indices(tokens, start,end)
            #token_start2,token_end2=find_closest_indices(tokens, start,end)
            token_start,token_end=getSpanToken_indices(tokens,span=entity1)
            token_start2,token_end2=getSpanToken_indices(tokens,span=entity2)
            start=tokens[token_start]["start"]
            end=tokens[token_end]["end"]
            start2=tokens[token_start2]["start"]
            end2=tokens[token_end2]["end"]
            dataDict["spans"].append({"start":start,"end":end,"token_start":token_start,"token_end":token_end,"label":selected_rows_iloc_rmvd["label1"][i],"text":entity1})
            dataDict["spans"].append({"start":start2,"end":end2,"token_start":token_start2,"token_end":token_end2,"label":selected_rows_iloc_rmvd["label2"][i],"text":entity2})
        else:
            l=l+1
            dict_list.append(dataDict)
            text=selected_rows_iloc_rmvd["sentence"][i]
            text=clean_text(text)
            entity1=clean_text(text=selected_rows_iloc_rmvd["entity1"][i])
            entity2=clean_text(text=selected_rows_iloc_rmvd["entity2"][i])
            sent_id_priv=selected_rows_iloc_rmvd["sent_id"][i]
            start,end=getEntity_start_end(text=text,entity=selected_rows_iloc_rmvd["entity1"][i])
            start2,end2=getEntity_start_end(text=text,entity=selected_rows_iloc_rmvd["entity2"][i])
            tokens=spacy_tokenizer(text=text)#split_text_with_positions(text=text)
            #token_start,token_end=find_closest_indices(tokens, start,end)
            #token_start2,token_end2=find_closest_indices(tokens, start2,end2)
            token_start,token_end=getSpanToken_indices(tokens,span=entity1)
            token_start2,token_end2=getSpanToken_indices(tokens,span=entity2)
            start=tokens[token_start]["start"]
            end=tokens[token_end]["end"]
            start2=tokens[token_start2]["start"]
            end2=tokens[token_end2]["end"]
            meta={"PMC_ID":selected_rows_iloc_rmvd["PMC_ID"][i],"Sentence_id":selected_rows_iloc_rmvd["sent_id"][i],"index":l-1,"line":l}
            dataDict={"meta":meta,"text":text,"tokens":tokens,"spans":[{"start":start,"end":end,"token_start":token_start,"token_end":token_end,"label":selected_rows_iloc_rmvd["label1"][i],"text":entity1},{"start":start2,"end":end2,"token_start":token_start2,"token_end":token_end2,"label":selected_rows_iloc_rmvd["label2"][i],"text":entity2}]}
    except:
        sent_id_priv=selected_rows_iloc_rmvd["sent_id"][i]
        #continue

  0%|          | 0/5296 [00:00<?, ?it/s]

100%|██████████| 5296/5296 [00:35<00:00, 147.42it/s]


In [140]:
sent_id_priv=selected_rows_iloc_rmvd["sent_id"][0]
dict_list=[]
text=selected_rows_iloc_rmvd["sentence"][0]
text=clean_text(text)
entity1=clean_text(text=selected_rows_iloc_rmvd["entity1"][0])
entity2=clean_text(text=selected_rows_iloc_rmvd["entity2"][0])
start,end=getEntity_start_end(text=text,entity=entity1)
start2,end2=getEntity_start_end(text=text,entity=entity2)
dataDict={"text":text,"spans":[{"start":start,"end":end,"label":selected_rows_iloc_rmvd["label1"][0]},{"start":start2,"end":end2,"label":selected_rows_iloc_rmvd["label2"][0]}]}
for i in tqdm(range(1,len(selected_rows_iloc_rmvd))):
    try:
        if sent_id_priv==selected_rows_iloc_rmvd["sent_id"][i] or text==clean_text(selected_rows_iloc_rmvd["sentence"][i]):
            entity1=clean_text(text=selected_rows_iloc_rmvd["entity1"][i])
            entity2=clean_text(text=selected_rows_iloc_rmvd["entity2"][i])
            start,end=getEntity_start_end(text=text,entity=entity1)
            start2,end2=getEntity_start_end(text=text,entity=entity2)
            dataDict["spans"].append({"start":start,"end":end,"label":selected_rows_iloc_rmvd["label1"][i],"text":entity1})
            dataDict["spans"].append({"start":start2,"end":end2,"label":selected_rows_iloc_rmvd["label2"][i],"text":entity2})
        else:
            dict_list.append(dataDict)
            text=selected_rows_iloc_rmvd["sentence"][i]
            text=clean_text(text)
            entity1=clean_text(text=selected_rows_iloc_rmvd["entity1"][i])
            entity2=clean_text(text=selected_rows_iloc_rmvd["entity2"][i])
            sent_id_priv=selected_rows_iloc_rmvd["sent_id"][i]
            start,end=getEntity_start_end(text=text,entity=selected_rows_iloc_rmvd["entity1"][i])
            start2,end2=getEntity_start_end(text=text,entity=selected_rows_iloc_rmvd["entity2"][i])
            dataDict={"text":text,"spans":[{"start":start,"end":end,"label":selected_rows_iloc_rmvd["label1"][i]},{"start":start2,"end":end2,"label":selected_rows_iloc_rmvd["label2"][i]}]}
    except:
        sent_id_priv=selected_rows_iloc_rmvd["sent_id"][i]

  0%|          | 0/5296 [00:00<?, ?it/s]

100%|██████████| 5296/5296 [00:00<00:00, 12581.15it/s]


In [22]:
len(dict_list)

4765

In [30]:
dict_list[3]

{'meta': {'PMC_ID': 'PMC10003408',
  'Sentence_id': 'PMC10003408_76',
  'index': 3,
  'line': 4},
 'text': 'Among the most common bacterial pathogens responsible for skin opportunistic infections, Staphylococcus spp.',
 'tokens': [{'text': 'Among', 'start': 0, 'end': 4, 'id': 0, 'ws': False},
  {'text': 'the', 'start': 6, 'end': 8, 'id': 1, 'ws': False},
  {'text': 'most', 'start': 10, 'end': 13, 'id': 2, 'ws': False},
  {'text': 'common', 'start': 15, 'end': 20, 'id': 3, 'ws': False},
  {'text': 'bacterial', 'start': 22, 'end': 30, 'id': 4, 'ws': False},
  {'text': 'pathogens', 'start': 32, 'end': 40, 'id': 5, 'ws': False},
  {'text': 'responsible', 'start': 42, 'end': 52, 'id': 6, 'ws': False},
  {'text': 'for', 'start': 54, 'end': 56, 'id': 7, 'ws': False},
  {'text': 'skin', 'start': 58, 'end': 61, 'id': 8, 'ws': False},
  {'text': 'opportunistic', 'start': 63, 'end': 75, 'id': 9, 'ws': False},
  {'text': 'infections', 'start': 77, 'end': 86, 'id': 10, 'ws': False},
  {'text': ',',

## Just Text

In [12]:

sent_id_priv=selected_rows_iloc_rmvd["sent_id"][0]
dict_list_text=[]
text=selected_rows_iloc_rmvd["sentence"][0]
text=clean_text(text)
l=1
meta={"PMC_ID":selected_rows_iloc_rmvd["PMC_ID"][0],"Sentence_id":selected_rows_iloc_rmvd["sent_id"][0],"index":0,"line":l}
dataDict={"meta":meta,"text":text}
for i in tqdm(range(1,len(selected_rows_iloc_rmvd))):
    try:
        if sent_id_priv==selected_rows_iloc_rmvd["sent_id"][i] or text==clean_text(selected_rows_iloc_rmvd["sentence"][i]):
            continue
        else:
            l=l+1
            dict_list_text.append(dataDict)
            text=selected_rows_iloc_rmvd["sentence"][i]
            text=clean_text(text)
            sent_id_priv=selected_rows_iloc_rmvd["sent_id"][i]
            meta={"PMC_ID":selected_rows_iloc_rmvd["PMC_ID"][i],"Sentence_id":selected_rows_iloc_rmvd["sent_id"][i],"index":l-1,"line":l}
            dataDict={"meta":meta,"text":text}
    except:
        sent_id_priv=selected_rows_iloc_rmvd["sent_id"][i]
        #continue

  0%|          | 0/5296 [00:00<?, ?it/s]

100%|██████████| 5296/5296 [00:00<00:00, 21311.27it/s]


In [13]:
dict_list_text[0]

{'meta': {'PMC_ID': 'PMC10000417',
  'Sentence_id': 'PMC10000417_144',
  'index': 0,
  'line': 1},
 'text': 'Chlorhexidine gel xanthan is a product indicated for the topical treatment of periodontitis and peri-implantitis.'}

In [22]:
dict_exemble={'text': 'The present study identified 32 genera that could cause opportunistic infections in humans.',
 'tokens': [{'text': 'The', 'start': 0, 'end': 3, 'id': 15, 'ws': True},
  {'text': 'present', 'start': 4, 'end': 11, 'id': 15, 'ws': True},
  {'text': 'study', 'start': 12, 'end': 17, 'id': 15, 'ws': True},
  {'text': 'identified', 'start': 18, 'end': 28, 'id': 15, 'ws': True},
  {'text': '32', 'start': 29, 'end': 31, 'id': 15, 'ws': True},
  {'text': 'genera', 'start': 32, 'end': 38, 'id': 15, 'ws': True},
  {'text': 'that', 'start': 39, 'end': 43, 'id': 15, 'ws': True},
  {'text': 'could', 'start': 44, 'end': 49, 'id': 15, 'ws': True},
  {'text': 'cause', 'start': 50, 'end': 55, 'id': 15, 'ws': True},
  {'text': 'opportunistic', 'start': 56, 'end': 69, 'id': 15, 'ws': True},
  {'text': 'infections', 'start': 70, 'end': 80, 'id': 15, 'ws': True},
  {'text': 'in', 'start': 81, 'end': 83, 'id': 15, 'ws': True},
  {'text': 'humans.', 'start': 84, 'end': 91, 'id': 15, 'ws': True}],
 'spans': [{'start': 56,
   'end': 79,
   'token_start': 9,
   'token_end': 10,
   'label': 'Disease',
   'text': 'opportunistic infections'},
  {'start': 84,
   'end': 89,
   'token_start': 9,
   'token_end': 10,
   'label': 'Species',
   'text': 'humans'}],
 'relation': []}

In [23]:
for span in dict_exemble["spans"]:
    print(span["text"]," : ",dict_exemble["text"][span["start"]:span["end"]+1])

opportunistic infections  :  opportunistic infections
humans  :  humans


In [31]:
for i in range(10):
    for span in dict_list[i]["spans"]:
        print(span["text"]," : ",dict_list[i]["text"][span["start"]:span["end"]+1])
    print()

Chlorhexidine  :  Chlorhexidine
periodontitis  :  periodontitis
Chlorhexidine  :  Chlorhexidine
periodontitis  :  periodontitis

opportunistic infections  :  opportunistic infections
humans  :  humans
opportunistic infections  :  opportunistic infections
humans  :  humans

skin lesions  :  skin lesions
Staphylococcus aureus  :  Staphylococcus aureus

skin opportunistic infections  :  skin opportunistic infections
Staphylococcus  :  Staphylococcus

Staphylococcal cutaneous infections  :  Staphylococcal cutaneous infections
Staphylococcal  :  Staphylococcal

Patients  :  Patients
CKD  :  CKD
Patients  :  Patients
CKD  :  CKD

Lactobacillus  :  Lactobacillus
ESKD  :  ESKD
Lactobacillus  :  Lactobacillus
ESKD  :  ESKD

phosphate  :  phosphate
CKD  :  CKD

HD  :  HD
phosphate  :  phosphate

rats  :  rats
hyper-ovulation  :  hyper-ovulation



In [18]:
text=selected_rows_iloc_rmvd["sentence"][0]

In [19]:
tokens=spacy_tokenizer(text=text)

In [21]:
tokens

[{'text': 'Chlorhexidine', 'start': 0, 'end': 12, 'id': 0, 'ws': True},
 {'text': 'gel', 'start': 14, 'end': 16, 'id': 1, 'ws': True},
 {'text': 'xanthan', 'start': 18, 'end': 24, 'id': 2, 'ws': True},
 {'text': 'is', 'start': 26, 'end': 27, 'id': 3, 'ws': True},
 {'text': 'a', 'start': 29, 'end': 29, 'id': 4, 'ws': True},
 {'text': 'product', 'start': 31, 'end': 37, 'id': 5, 'ws': True},
 {'text': 'indicated', 'start': 39, 'end': 47, 'id': 6, 'ws': True},
 {'text': 'for', 'start': 49, 'end': 51, 'id': 7, 'ws': True},
 {'text': 'the', 'start': 53, 'end': 55, 'id': 8, 'ws': True},
 {'text': 'topical', 'start': 57, 'end': 63, 'id': 9, 'ws': True},
 {'text': 'treatment', 'start': 65, 'end': 73, 'id': 10, 'ws': True},
 {'text': 'of', 'start': 75, 'end': 76, 'id': 11, 'ws': True},
 {'text': 'periodontitis', 'start': 78, 'end': 90, 'id': 12, 'ws': True},
 {'text': 'and', 'start': 92, 'end': 94, 'id': 13, 'ws': True},
 {'text': 'peri', 'start': 96, 'end': 99, 'id': 14, 'ws': False},
 {'text':

In [32]:
t=0
with open(os.path.join(cf.PATH_RESULTS,"Cleaned_Data_Spacy_Tokens_All_10_cleaned_tokens_no_ws.jsonl"), 'w') as file:
    for item in dict_list:
        if t<10:
            t=t+1
            json.dump(item, file)
            file.write('\n')
        else:
            break

In [33]:
jsonData=[]
with open(os.path.join(cf.PATH_RESULTS,"Cleaned_Data_Spacy_Tokens_All_10_cleaned_tokens_no_ws.jsonl"), 'r') as file:
    for item in jsonlines.Reader(file):
        jsonData.append(item)

In [37]:
#get json data from jsonl file with out the tokens key
jsonData=[]
with open(os.path.join(cf.PATH_RESULTS,"Cleaned_Data_Spacy_Tokens_All_10_cleaned_tokens_no_ws.jsonl"), 'r') as file:
    for item in jsonlines.Reader(file):
        item.pop("tokens")
        jsonData.append(item)

In [43]:
jsonData[0]

{'meta': {'PMC_ID': 'PMC10000417',
  'Sentence_id': 'PMC10000417_144',
  'index': 0,
  'line': 1},
 'text': 'Chlorhexidine gel xanthan is a product indicated for the topical treatment of periodontitis and peri-implantitis.',
 'spans': [{'start': 0,
   'end': 12,
   'token_start': 0,
   'token_end': 0,
   'label': 'Chemical',
   'text': 'Chlorhexidine'},
  {'start': 78,
   'end': 90,
   'token_start': 12,
   'token_end': 12,
   'label': 'Disease',
   'text': 'periodontitis'},
  {'start': 0,
   'end': 12,
   'token_start': 0,
   'token_end': 0,
   'label': 'Chemical',
   'text': 'Chlorhexidine'},
  {'start': 78,
   'end': 90,
   'token_start': 12,
   'token_end': 12,
   'label': 'Disease',
   'text': 'periodontitis'}]}

In [44]:
with open(os.path.join(cf.PATH_RESULTS,"Cleaned_Data_Spacy_Tokens_All_10_cleaned_no_tokens_no_ws.jsonl"), "w") as f:
    for item in jsonData:
        json.dump(item, f)
        f.write('\n')

In [38]:
jsonData[0]

{'meta': {'PMC_ID': 'PMC10000417',
  'Sentence_id': 'PMC10000417_144',
  'index': 0,
  'line': 1},
 'text': 'Chlorhexidine gel xanthan is a product indicated for the topical treatment of periodontitis and peri-implantitis.',
 'spans': [{'start': 0,
   'end': 12,
   'token_start': 0,
   'token_end': 0,
   'label': 'Chemical',
   'text': 'Chlorhexidine'},
  {'start': 78,
   'end': 90,
   'token_start': 12,
   'token_end': 12,
   'label': 'Disease',
   'text': 'periodontitis'},
  {'start': 0,
   'end': 12,
   'token_start': 0,
   'token_end': 0,
   'label': 'Chemical',
   'text': 'Chlorhexidine'},
  {'start': 78,
   'end': 90,
   'token_start': 12,
   'token_end': 12,
   'label': 'Disease',
   'text': 'periodontitis'}]}

In [ ]:
no_ws_list=[",",".","-","(",")","[","]"]

In [25]:
dict_list[0:10]

[{'meta': {'PMC_ID': 'PMC10000417',
   'Sentence_id': 'PMC10000417_144',
   'index': 0,
   'line': 1},
  'text': 'Chlorhexidine gel xanthan is a product indicated for the topical treatment of periodontitis and peri-implantitis.',
  'tokens': [{'text': 'Chlorhexidine',
    'start': 0,
    'end': 12,
    'id': 0,
    'ws': True},
   {'text': 'gel', 'start': 14, 'end': 16, 'id': 1, 'ws': True},
   {'text': 'xanthan', 'start': 18, 'end': 24, 'id': 2, 'ws': True},
   {'text': 'is', 'start': 26, 'end': 27, 'id': 3, 'ws': True},
   {'text': 'a', 'start': 29, 'end': 29, 'id': 4, 'ws': True},
   {'text': 'product', 'start': 31, 'end': 37, 'id': 5, 'ws': True},
   {'text': 'indicated', 'start': 39, 'end': 47, 'id': 6, 'ws': True},
   {'text': 'for', 'start': 49, 'end': 51, 'id': 7, 'ws': True},
   {'text': 'the', 'start': 53, 'end': 55, 'id': 8, 'ws': True},
   {'text': 'topical', 'start': 57, 'end': 63, 'id': 9, 'ws': True},
   {'text': 'treatment', 'start': 65, 'end': 73, 'id': 10, 'ws': True}

In [62]:
selected_rows_iloc_rmvd.sent_id.value_counts()

sent_id
PMC5260789_864     3
PMC5517471_18      3
PMC3153137_319     3
PMC3202864_45      3
PMC5103241_2468    3
                  ..
PMC4877354_678     1
PMC4877354_688     1
PMC4877746_45      1
PMC4877746_130     1
PMC5868825_140     1
Name: count, Length: 4766, dtype: int64

In [115]:
getEntity_start_end(text=selected_rows_iloc_rmvd["sentence"][18].strip().replace("( ","(").replace(" )",")"),entity=selected_rows_iloc_rmvd["entity1"][18].strip())

In [116]:
selected_rows_iloc_rmvd["entity1"][18].strip()

'Ca2+'

In [113]:
selected_rows_iloc_rmvd["sentence"][18].strip().replace("( ","(").replace(" )",")")

'reported that curcumin influenced Ca2 + homeostasis in ovarian cancer cells [ 103 ] .'

In [75]:
selected_rows_iloc_rmvd["sent_id"][1]

'PMC10000417_220'

In [76]:
selected_rows_iloc_rmvd

,level_0,index,PMC_ID,sent_id,sentence,entity1,entity2,label1,label2
0,5,78,PMC10000417,PMC10000417_144,Chlorhexidine gel xanthan is a product indicat...,Chlorhexidine,periodontitis,Chemical,Disease
1,14,128,PMC10000417,PMC10000417_220,Chlorhexidine gel xanthan is a product indicat...,Chlorhexidine,periodontitis,Chemical,Disease
3,106,525,PMC10002314,PMC10002314_214,The present study identified 32 genera that co...,opportunistic infections,humans,Disease,Species
4,114,551,PMC10002314,PMC10002314_286,The present study identified 32 genera that co...,opportunistic infections,humans,Disease,Species
5,182,880,PMC10003408,PMC10003408_36,The cultures of pus from skin lesions confirme...,skin lesions,Staphylococcus aureus,Disease,Species
...,...,...,...,...,...,...,...,...,...
6725,63364,230065,PMC5862454,PMC5862454_340,Low intake was associated with higher prevalen...,caries,S. mutans,Disease,Species
6729,63490,230343,PMC5866915,PMC5866915_130,( 2006 ) showed that HPV infection is less pre...,HPV infection,HPV,Disease,Species
6730,63501,230356,PMC5866915,PMC5866915_149,We detected a negative association between oro...,oropharyngeal HPV infection,HPV,Disease,Species
6731,63507,230366,PMC5866915,PMC5866915_160,"Koilocitosis , a marker of nuclear changes con...",HPV infection,HPV,Disease,Species


In [1]:
import os

In [44]:
data_list=[]
with open("/home/aberhe/Projects/SANTAL/ASLR/annotation_guideline_examples_annotated.jsonl", 'r') as json_file:
    for line in json_file:
        try:
            data = json.loads(line)
            data_list.append(data)
        except json.JSONDecodeError as e:
            print(f"Error loading JSON: {e}")


In [46]:
data_list

[{'text': 'The bacterium Pseudomonas putida degrades chlorinated hydrocarbons, including trichloroethylene, a common groundwater pollutant, through enzymatic processes.',
  'relation': 'Degrades',
  '_input_hash': 148995372,
  '_task_hash': 623521628,
  '_is_binary': False,
  'spans': [{'start': 14,
    'end': 32,
    'token_start': 2,
    'token_end': 3,
    'label': 'Species'},
   {'start': 42,
    'end': 66,
    'token_start': 5,
    'token_end': 6,
    'label': 'Chemical'},
   {'start': 78,
    'end': 95,
    'token_start': 9,
    'token_end': 9,
    'label': 'Chemical'}],
  'tokens': [{'text': 'The',
    'start': 0,
    'end': 3,
    'id': 0,
    'ws': True,
    'disabled': False},
   {'text': 'bacterium',
    'start': 4,
    'end': 13,
    'id': 1,
    'ws': True,
    'disabled': False},
   {'text': 'Pseudomonas',
    'start': 14,
    'end': 25,
    'id': 2,
    'ws': True,
    'disabled': False},
   {'text': 'putida',
    'start': 26,
    'end': 32,
    'id': 3,
    'ws': True,


In [45]:
len(data_list)

1

In [9]:
clean_data_list=[]
for i in range (len(data_list)):
    if "++" not in data_list[i]["text"] and len(data_list[i]["text"].split())<50:
        print(i)
        print(data_list[i]["text"].strip())
        clean_data_list.append(data_list[i])

2
Identified taxa of the oronasal habitat included Bergeyella spp. , Dialister spp. , Fusobacterium nucleatum , Oribacterium sinus , Peptostreptococcus oralis , Prevotella oris , Rothia dentocariosa , Streptococcus oralis , Streptococcus salivarius , Veillonella parvula , and Veillonella spp .
4
Real-time PCR analysis was performed by using Primerdesign  Ltd . ( Great Britain ) genesis standards kits for Lactobacillus acidophilus , Streptococcus sanguinis , Porphyromonas gingivalis , S. mutans , Actinomyces israelli , Prevotella melaninogenica , Fusobacterium nucleatum , Peptostreptococcus micros , P. endodontalis , and P. gingivalis .
6
* In parenthesis , species that increased their respiratory activity with the indicated metabolite : Smut , Streptococcus mutans ; Aa , Aggregatibacter actinomycetemcomitans ; Ssob , Streptococcus sobrinus ; Pi , Prevotella intermedia ; Pg , Porphyromonas gingivalis ; Tf , Tannerella forsythia .
10
Real-time PCR analysis was performed by using Primerde

In [ ]:
clean_data_list[0]

{'text': 'Identified taxa of the oronasal habitat included Bergeyella spp. , Dialister spp. , Fusobacterium nucleatum , Oribacterium sinus , Peptostreptococcus oralis , Prevotella oris , Rothia dentocariosa , Streptococcus oralis , Streptococcus salivarius , Veillonella parvula , and Veillonella spp .',
 'Sent_ID': 'PMC3797391_87',
 'Entities': {'text': ['Dialister spp.',
   'Fusobacterium nucleatum',
   'Oribacterium sinus',
   'Peptostreptococcus oralis',
   'Prevotella oris',
   'Rothia dentocariosa',
   'Streptococcus oralis',
   'Streptococcus salivarius',
   'Veillonella parvula',
   'Veillonella spp'],
  'labels': ['Species',
   'Species',
   'Species',
   'Species',
   'Species',
   'Species',
   'Species',
   'Species',
   'Species',
   'Species'],
  'start_chuncks': [66, 82, 107, 127, 154, 171, 192, 214, 240, 265],
  'end_chuncks': [80, 105, 125, 152, 169, 190, 212, 238, 259, 280]}}

In [13]:
len(clean_data_list)

4282

In [14]:
 len(clean_data_list[0]["Entities"])

4

In [18]:
for e in enumerateclean_data_list[ind]["Entities"]["text"]:
    print(e)

Dialister spp.
Fusobacterium nucleatum
Oribacterium sinus
Peptostreptococcus oralis
Prevotella oris
Rothia dentocariosa
Streptococcus oralis
Streptococcus salivarius
Veillonella parvula
Veillonella spp


In [15]:
clean_data_list[0]["Entities"]

{'text': ['Dialister spp.',
  'Fusobacterium nucleatum',
  'Oribacterium sinus',
  'Peptostreptococcus oralis',
  'Prevotella oris',
  'Rothia dentocariosa',
  'Streptococcus oralis',
  'Streptococcus salivarius',
  'Veillonella parvula',
  'Veillonella spp'],
 'labels': ['Species',
  'Species',
  'Species',
  'Species',
  'Species',
  'Species',
  'Species',
  'Species',
  'Species',
  'Species'],
 'start_chuncks': [66, 82, 107, 127, 154, 171, 192, 214, 240, 265],
 'end_chuncks': [80, 105, 125, 152, 169, 190, 212, 238, 259, 280]}

In [12]:
print("hello")

hello


In [1]:
"""f=0
for i in range(0,len(clean_data_list),1000):
    print(i)
    with open(os.path.join(cf.PATH_RESULTS,'Cleaned_data_for_annotation_'+str(i)+'.jsonl'), "w") as file:
        data={}
        for ind in range(f,i):
            data["text"]=clean_data_list[ind]["text"]
            data["Sent_ID"]=clean_data_list[ind]["Sent_ID"]  
            spans=[{"text":clean_data_list[ind]["Entities"]["text"][e],"start":clean_data_list[ind]["Entities"]["start_chuncks"][e],"end":clean_data_list[ind]["Entities"]["end_chuncks"][e],"label":clean_data_list[ind]["Entities"]["labels"][e]} for e in len(clean_data_list[ind]["Entities"]["text"])] 
            data["spans"]=spans
            json.dump(data, file)
            file.write("\n")
        f=i
"""

'f=0\nfor i in range(0,len(clean_data_list),1000):\n    print(i)\n    with open(os.path.join(cf.PATH_RESULTS,\'Cleaned_data_for_annotation_\'+str(i)+\'.jsonl\'), "w") as file:\n        data={}\n        for ind in range(f,i):\n            data["text"]=clean_data_list[ind]["text"]\n            data["Sent_ID"]=clean_data_list[ind]["Sent_ID"]  \n            spans=[{"text":clean_data_list[ind]["Entities"]["text"][e],"start":clean_data_list[ind]["Entities"]["start_chuncks"][e],"end":clean_data_list[ind]["Entities"]["end_chuncks"][e],"label":clean_data_list[ind]["Entities"]["labels"][e]} for e in len(clean_data_list[ind]["Entities"]["text"])] \n            data["spans"]=spans\n            json.dump(data, file)\n            file.write("\n")\n        f=i\n'

In [66]:
data_list=[]
unique_sentences=[]
data_list_s=[]
for i in ["1000","2000","3000","4000"]:
    with open(os.path.join(cf.PATH_RESULTS,'Cleaned_data_for_annotation_'+str(i)+'.jsonl'), 'r') as json_file:
        c=0
        for line in json_file:
            c=c+1
            try:
                data = json.loads(line)
                if data["text"] not in unique_sentences:
                    unique_sentences.append(data["text"])
                    data_list.append(data)
                    data_list_s.append(data)
                else:
                    if data_list[unique_sentences.index(data["text"])]["spans"]!=data["spans"]:
                        tmp=data_list[unique_sentences.index(data["text"])]["spans"]
                        not_intmp=[]
                        print(tmp)
                        print(data["spans"])
                        for span in data["spans"]:
                            if span not in tmp:
                                tmp.append(span)
                                not_intmp.append(span)
                        data_list[unique_sentences.index(data["text"])]["spans"]=tmp
                        print(i,unique_sentences.index(data["text"]),"Not Equal",not_intmp)
                        print(not_intmp)
                        print(data["spans"])
                    else:
                        continue
            except json.JSONDecodeError as e:
                print(f"Error loading JSON: {e}")

[{'start': 50, 'end': 59, 'label': 'Disease'}, {'start': 83, 'end': 119, 'label': 'Species'}, {'start': 121, 'end': 141, 'label': 'Species'}, {'start': 143, 'end': 165, 'label': 'Species'}, {'start': 167, 'end': 186, 'label': 'Species'}, {'start': 188, 'end': 209, 'label': 'Species'}, {'start': 211, 'end': 235, 'label': 'Species'}, {'start': 240, 'end': 266, 'label': 'Species'}]
[{'start': 49, 'end': 58, 'label': 'Disease'}, {'start': 82, 'end': 118, 'label': 'Species'}, {'start': 120, 'end': 140, 'label': 'Species'}, {'start': 142, 'end': 164, 'label': 'Species'}, {'start': 166, 'end': 185, 'label': 'Species'}, {'start': 187, 'end': 208, 'label': 'Species'}, {'start': 210, 'end': 234, 'label': 'Species'}, {'start': 239, 'end': 265, 'label': 'Species'}]
1000 203 Not Equal [{'start': 49, 'end': 58, 'label': 'Disease'}, {'start': 82, 'end': 118, 'label': 'Species'}, {'start': 120, 'end': 140, 'label': 'Species'}, {'start': 142, 'end': 164, 'label': 'Species'}, {'start': 166, 'end': 185, 

In [39]:
unique_sentences.index(data_list[7]["text"])

7

In [15]:
len(data_list)

2221

In [73]:
data_list[250]

{'text': 'Background Whooping cough is a highly contagious acute infectious disease caused by bacilli of the genus Bordetella pertussis , parapertussis and extremely rare in humans B. bronchiseptica with paroxysmal cough and variable impaired general condition , hematological changes and high risk of complications .',
 'Sent_ID': 'PMC5103241_90',
 'spans': [{'start': 12, 'end': 26, 'label': 'Disease'},
  {'start': 50, 'end': 74, 'label': 'Disease'},
  {'start': 106, 'end': 126, 'label': 'Species'},
  {'start': 164, 'end': 170, 'label': 'Species'},
  {'start': 171, 'end': 188, 'label': 'Species'},
  {'start': 194, 'end': 210, 'label': 'Disease'},
  {'start': 224, 'end': 250, 'label': 'Disease'},
  {'start': 13, 'end': 27, 'label': 'Disease'},
  {'start': 51, 'end': 75, 'label': 'Disease'},
  {'start': 107, 'end': 127, 'label': 'Species'},
  {'start': 165, 'end': 171, 'label': 'Species'},
  {'start': 172, 'end': 189, 'label': 'Species'},
  {'start': 195, 'end': 211, 'label': 'Disease'},


In [74]:
data_list_s[250]

{'text': 'Background Whooping cough is a highly contagious acute infectious disease caused by bacilli of the genus Bordetella pertussis , parapertussis and extremely rare in humans B. bronchiseptica with paroxysmal cough and variable impaired general condition , hematological changes and high risk of complications .',
 'Sent_ID': 'PMC5103241_90',
 'spans': [{'start': 12, 'end': 26, 'label': 'Disease'},
  {'start': 50, 'end': 74, 'label': 'Disease'},
  {'start': 106, 'end': 126, 'label': 'Species'},
  {'start': 164, 'end': 170, 'label': 'Species'},
  {'start': 171, 'end': 188, 'label': 'Species'},
  {'start': 194, 'end': 210, 'label': 'Disease'},
  {'start': 224, 'end': 250, 'label': 'Disease'},
  {'start': 13, 'end': 27, 'label': 'Disease'},
  {'start': 51, 'end': 75, 'label': 'Disease'},
  {'start': 107, 'end': 127, 'label': 'Species'},
  {'start': 165, 'end': 171, 'label': 'Species'},
  {'start': 172, 'end': 189, 'label': 'Species'},
  {'start': 195, 'end': 211, 'label': 'Disease'},


In [54]:
data_list_s[3]

{'text': 'The orange cluster ( Fusobacterium nucleatum subspecies , Prevotella intermedia , Prevotella nigrescens , Peptostreptococcus micros , Campylobacter rectus , Campylobacter showae , Campylobacter gracilis , Eubacterium nodatum , Streptococcus constellatus , Fusobacterium periodonticum ) .',
 'Sent_ID': 'PMC3703280_144',
 'spans': [{'start': 20, 'end': 43, 'label': 'Species'},
  {'start': 56, 'end': 77, 'label': 'Species'},
  {'start': 79, 'end': 100, 'label': 'Species'},
  {'start': 102, 'end': 127, 'label': 'Species'},
  {'start': 129, 'end': 149, 'label': 'Species'},
  {'start': 151, 'end': 171, 'label': 'Species'},
  {'start': 173, 'end': 195, 'label': 'Species'},
  {'start': 197, 'end': 216, 'label': 'Species'},
  {'start': 218, 'end': 244, 'label': 'Species'},
  {'start': 246, 'end': 273, 'label': 'Species'}]}

In [103]:
data_spacy_list=[]
with open(os.path.join(cf.PATH_RESULTS,'Cleaned_Data_Spacy_Tokens_All_just_text_10.jsonl'), 'r') as file:
    for line in file:
        data = json.loads(line)
        data_spacy_list.append(data)
print(len(data_spacy_list))

3973


In [104]:
data_spacy_list[0]

{'meta': {'PMC_ID': 'PMC10000417',
  'Sentence_id': 'PMC10000417_144',
  'index': 0,
  'line': 1},
 'text': 'Chlorhexidine gel xanthan is a product indicated for the topical treatment of periodontitis and peri-implantitis.'}

In [110]:
data_spacy=[]
unique_sentences=[]
best_example=[]
with open(os.path.join(cf.PATH_RESULTS,'Cleaned_Data_Spacy_Tokens_All_10_ws_t_f.jsonl'), 'r') as file:
    for line in file:
        data = json.loads(line)
        if data["text"] not in unique_sentences:
            unique_sentences.append(data["text"])
            data_spacy.append(data)

            if len(data["text"].split(","))<2:
                best_example.append(data)
        else:
            continue
print(len(data_spacy),len(best_example)) 

1837 1195


In [111]:
#randmly devide the data into 4 folds data_spacy
import random
random.shuffle(data_spacy)
len(data_spacy)
fold=4
fold_size=int(len(data_spacy)/fold)
fold_1=data_spacy[0:fold_size]
fold_2=data_spacy[fold_size:fold_size*2]
fold_3=data_spacy[fold_size*2:fold_size*3]
fold_4=data_spacy[fold_size*3:]

print(len(fold_1),len(fold_2),len(fold_3),len(fold_4))


459 459 459 460


In [112]:
random.shuffle(best_example)

In [114]:

examples_for_all_folds=best_example[0:100]


In [117]:
#Save the hundred examples as jsonl files
with open(os.path.join(cf.PATH_RESULTS,'Examples_100.jsonl'), "w") as file:
    for item in examples_for_all_folds:
        json.dump(item, file)
        file.write('\n')


In [115]:
#Add the 100 examples to each fold if the sentence is not in the fold
for i in range(len(examples_for_all_folds)):
    if examples_for_all_folds[i] not in fold_1:
        fold_1.append(examples_for_all_folds[i])
    if examples_for_all_folds[i] not in fold_2:
        fold_2.append(examples_for_all_folds[i])
    if examples_for_all_folds[i] not in fold_3:
        fold_3.append(examples_for_all_folds[i])
    if examples_for_all_folds[i] not in fold_4:
        fold_4.append(examples_for_all_folds[i])
    else:
        continue
print(len(fold_1),len(fold_2),len(fold_3),len(fold_4))

527 529 542 539


In [116]:
with open(os.path.join(cf.PATH_RESULTS,'Cleaned_Data_Spacy_Tokens_Fold_1.jsonl'), 'w') as file:
    for item in fold_1:
        json.dump(item, file)
        file.write('\n')

with open(os.path.join(cf.PATH_RESULTS,'Cleaned_Data_Spacy_Tokens_Fold_2.jsonl'), 'w') as file:
    for item in fold_2:
        json.dump(item, file)
        file.write('\n')

with open(os.path.join(cf.PATH_RESULTS,'Cleaned_Data_Spacy_Tokens_Fold_3.jsonl'), 'w') as file:
    for item in fold_3:
        json.dump(item, file)
        file.write('\n')

with open(os.path.join(cf.PATH_RESULTS,'Cleaned_Data_Spacy_Tokens_Fold_4.jsonl'), 'w') as file:
    for item in fold_4:
        json.dump(item, file)
        file.write('\n')

In [97]:
#Write list of dictionaries to jsonl file
with open(os.path.join(cf.PATH_RESULTS,'Cleaned_Data_Spacy_Tokens_All_10_ws_t_f_unique_sentences.jsonl'), 'w') as file:
    for item in data_spacy:
        json.dump(item, file)
        file.write('\n')

In [98]:
len(data_spacy)

1837

In [75]:
os.listdir(cf.PATH_RESULTS)

['Bacterias_Articles_Adjacency_Matrix.csv',
 'unique_included_article_files.txt',
 'extracted_2_Entities.csv',
 'extracted_2_Entities_with_Relation.csv',
 'extracted_2_Entities_04_09.csv',
 'Species_Disease_extracted_2_Entities_04_09.csv',
 'Species_Disease_extracted_2_Entities_04_09_For_RE.csv',
 'Sentences for species and diseases NER and RE annotation.json',
 'cleaned.json',
 'BioNER',
 '.ipynb_checkpoints',
 'Included_articles',
 'Bacterias_Species_Articles_Adjacency_Matrix_103.csv',
 'unique_included_article_files_Bacteria_Species.txt',
 'Bacterias_Articles_Adjacency_Matrix_72.csv',
 'Bacterias_Articles_Adjacency_Matrix_127.csv',
 'Bacterias_Articles_Adjacency_Matrix_127_withCounts.csv',
 'species_annotation.jsonl.json',
 'species_annotation.jsonl',
 'Cleaned_data_for_annotation.jsonl',
 'annotation_guideline_examples.jsonl',
 'Cleaned_data_for_annotation_1000.jsonl',
 'Cleaned_data_for_annotation_2000.jsonl',
 'Cleaned_data_for_annotation_3000.jsonl',
 'Cleaned_data_for_annotatio

### Annotation Guideline Examples

In [ ]:
annotation_guideline={}
sdi_labels=["Associtation_Pos","Associtation_Neg","Neutral"]
ssi_labels=["Comptetes","Co-Exits","Inhabits","Produces","Assosation_Pos","Association_Neg"]
sci_labels=["Produces","Consumes","Degrades"]
sdi=["An increased abundance of Akkermansia muciniphila in cirrhosis patients is linked to improved outcomes, as it is associated with reduced inflammation and enhanced gut barrier function [10].",
     "Higher levels of pathogenic bacteria like Escherichia coli in the gut microbiome of cirrhosis patients are correlated with disease progression, contributing to systemic inflammation and complications (Bajaj et al., 2018).",
     "The presence of Lactobacillus rhamnosus in the gut microbiomes of cirrhosis patients often lacks a distinct impact on disease progression, requiring further investigation."]
ssi=["In the gut microbiome, the bacterial species Bacteroides thetaiotaomicron and Firmicutes often compete for access to complex carbohydrates, demonstrating a competitive relationship for these resources.",
     "Within the human gut microbiome, various bacterial species, such as Bifidobacterium breve and Escherichia coli, coexist, each playing distinct roles in the fermentation of dietary fibers and the maintenance of microbial balance.",
     "Some gut bacteria, like Clostridium difficile, can inhibit the growth of other bacterial species by producing toxins that disrupt the balance of the microbiome, resulting in potential infections and dysbiosis.",
     "Certain gut bacteria, such as Akkermansia muciniphila, consume mucin glycoproteins in the intestinal lining, contributing to mucin degradation and the utilization of this energy source."]
sci=["Methanogenic archaea, such as Methanobrevibacter smithii, produce methane gas during anaerobic digestion in the gut, contributing to its presence in the digestive system [7].",
     "Grazing herbivores, like the plains zebra (Equus zebra), consume grasses rich in secondary metabolites like tannins, which can affect both their diet choices and the availability of these chemical compounds.",
     "The bacterium Pseudomonas putida degrades chlorinated hydrocarbons, including trichloroethylene, a common groundwater pollutant, through enzymatic processes."]

In [ ]:
list_ann_guideline=[]
for i, sent in enumerate(sdi):
    annotation_guideline={}
    annotation_guideline["text"]=sent
    annotation_guideline["relation"]=sdi_labels[i]
    list_ann_guideline.append(annotation_guideline)
for i, sent in enumerate(ssi):
    annotation_guideline={}
    annotation_guideline["text"]=sent
    annotation_guideline["relation"]=ssi_labels[i]
    list_ann_guideline.append(annotation_guideline)
for i, sent in enumerate(sci):
    annotation_guideline={}
    annotation_guideline["text"]=sent
    annotation_guideline["relation"]=sci_labels[i]
    list_ann_guideline.append(annotation_guideline)

In [16]:
list_ann_guideline

[{'text': 'An increased abundance of Akkermansia muciniphila in cirrhosis patients is linked to improved outcomes, as it is associated with reduced inflammation and enhanced gut barrier function [10].',
  'relation': 'Associtation_pos'},
 {'text': 'Higher levels of pathogenic bacteria like Escherichia coli in the gut microbiome of cirrhosis patients are correlated with disease progression, contributing to systemic inflammation and complications (Bajaj et al., 2018).',
  'relation': 'Associtation_neg'},
 {'text': 'The presence of Lactobacillus rhamnosus in the gut microbiomes of cirrhosis patients often lacks a distinct impact on disease progression, requiring further investigation.',
  'relation': 'Neutral'},
 {'text': 'In the gut microbiome, the bacterial species Bacteroides thetaiotaomicron and Firmicutes often compete for access to complex carbohydrates, demonstrating a competitive relationship for these resources.',
  'relation': 'Comptetes'},
 {'text': 'Within the human gut microb

In [12]:
with open(os.path.join(cf.PATH_RESULTS,'annotation_guideline_examples.jsonl'), "w") as file:
    for data in list_ann_guideline:
        json.dump(data, file)
        file.write("\n")

In [21]:
#preparing group annootation data
data_list=[]
with open(os.path.join(cf.PATH_RESULTS,'Entities_Annotation_All_Greater_than_2_less_Than_4_1000_Example.jsonl'), 'r') as file:
    for line in file:
        data = json.loads(line)
        data_list.append(data)
print(len(data_list))

32466


In [22]:
data_list[1]

{'meta': {'PMC_ID': 'PMC5617869',
  'Sentence_id': 'PMC5617869_286',
  'index': 11124,
  'line': 11125,
  'human': 'S',
  'infections': 'D',
  'periodontal diseases': 'D'},
 'text': 'It is recognized to be associated with human infections [ 26 , 36 , 45 – 47 ] , and are able to survive within oral epithelial cells as well as to modulate the epithelial immune response , leading to initiation and progression of periodontal diseases [ 25 ] .'}

In [10]:
#preparing group annootation data
data_list_atd=[]
with open(os.path.join(cf.PATH_RESULTS,'example_1000_EB.jsonl'), 'r') as file:
    for line in file:
        data = json.loads(line)
        data_list_atd.append(data)
print(len(data_list_atd))

556


In [12]:
data_list_atd[1]

{'meta': {'PMC_ID': 'PMC5617869',
  'Sentence_id': 'PMC5617869_286',
  'index': 11124,
  'line': 11125,
  'human': 'S',
  'infections': 'D',
  'periodontal diseases': 'D'},
 'text': 'It is recognized to be associated with human infections [ 26 , 36 , 45 – 47 ] , and are able to survive within oral epithelial cells as well as to modulate the epithelial immune response , leading to initiation and progression of periodontal diseases [ 25 ] .',
 '_input_hash': 1778720875,
 '_task_hash': -1773008493,
 '_is_binary': False,
 'spans': [{'start': 111,
   'end': 132,
   'token_start': 25,
   'token_end': 27,
   'label': 'Habitat'},
  {'start': 230,
   'end': 250,
   'token_start': 44,
   'token_end': 45,
   'label': 'Disease'}],
 'tokens': [{'text': 'It',
   'start': 0,
   'end': 2,
   'id': 0,
   'ws': True,
   'disabled': False},
  {'text': 'is', 'start': 3, 'end': 5, 'id': 1, 'ws': True, 'disabled': False},
  {'text': 'recognized',
   'start': 6,
   'end': 16,
   'id': 2,
   'ws': True,
   'd

In [19]:
data_list_atd[0].keys()

dict_keys(['meta', 'text', '_input_hash', '_task_hash', '_is_binary', 'spans', 'tokens', '_view_id', 'relations', 'answer', '_timestamp', '_annotator_id', '_session_id'])

In [20]:
data_list[0].keys()

dict_keys(['meta', 'text'])

In [27]:
#create the group annotation data, 100 examples, from the data_list where sentnece id is equal to the sentence id in data_list_atd
group_annotation_data=[]
for i in range(100):
    #for j in range(len(data_list)):
    if data_list_atd[i]["meta"]["Sentence_id"]==data_list[i+1]["meta"]["Sentence_id"]:
        group_annotation_data.append(data_list[i])
    elif data_list_atd[i]["meta"]["Sentence_id"]==data_list[i]["meta"]["Sentence_id"]:
        group_annotation_data.append(data_list[i])
    else:
        print("correct the sentence id")
        print(data_list_atd[i]["meta"]["Sentence_id"],data_list[i]["meta"]["Sentence_id"])
        continue


In [28]:
len(group_annotation_data)

100

In [32]:
#Save the hundred examples as jsonl files
with open(os.path.join(cf.PATH_RESULTS,'Group_4_Annotation_100.jsonl'), "w") as file:
    for item in group_annotation_data:
        json.dump(item, file)
        file.write('\n')